In [1]:
%load_ext autoreload
%autoreload 2

import torch.functional as F
import dense_correspondence_manipulation.utils.utils as utils
utils.add_dense_correspondence_to_python_path()
from dense_correspondence.training.training import *
import sys
import os
import logging

#utils.set_default_cuda_visible_devices()
utils.set_cuda_visible_devices([0]) # use this to manually set CUDA_VISIBLE_DEVICES

from dense_correspondence.training.training import DenseCorrespondenceTraining
from dense_correspondence.dataset.spartan_dataset_masked import SpartanDataset
logging.basicConfig(level=logging.INFO)

from dense_correspondence.evaluation.evaluation import DenseCorrespondenceEvaluation

setting CUDA_VISIBLE_DEVICES =  0,


## Load the configuration for training

In [2]:
project_dir = utils.getDenseCorrespondenceSourceDir()
dataset_config_filename = os.path.join(project_dir, 'data/pdc/trained_models/shoes_consistent_M_background_0.500_3/dataset.yaml')
dataset_config = utils.getDictFromYamlFilename(dataset_config_filename)

In [3]:
dataset = SpartanDataset(config_expanded=dataset_config)

Using SpartanDataset:
   - in train mode
   - number of scenes 16
   - total images:     5849


In [4]:
train_config_file = os.path.join(project_dir, 'data/pdc/trained_models/shoes_consistent_M_background_0.500_3/training.yaml')

train_config = utils.getDictFromYamlFilename(train_config_file)

In [5]:
logging_dir = "data/pdc/trained_models/my_models"
num_iterations = 3500
d = 3 # the descriptor dimension
name = "shoes_training_%d_aploss" %(d)
train_config["training"]["logging_dir_name"] = name
train_config["training"]["logging_dir"] = logging_dir
train_config["dense_correspondence_network"]["descriptor_dimension"] = d
train_config["training"]["num_iterations"] = num_iterations

TRAIN = True
EVALUATE = True

## Train the network

This should take about ~12-15 minutes with a GTX 1080 Ti

In [ ]:
# All of the saved data for this network will be located in the
# code/data/pdc/trained_models/tutorials/caterpillar_3 folder

if False:
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
if TRAIN:
    print "training descriptor of dimension %d" %(d)
    train = DenseCorrespondenceTraining(dataset=dataset, config=train_config)
    train.run()
    print "finished training descriptor of dimension %d" %(d)


training descriptor of dimension 3
https://ui.neptune.ai/general-dense-object-nets/general-dense-object-nets/e/SAN-46
using SINGLE_OBJECT_WITHIN_SCENE
logging_dir: /home/tomasz/code/data/pdc/trained_models/my_models/shoes_training_3_aploss
('Saving to params to file: ', '/home/tomasz/code/data/pdc/trained_models/my_models/shoes_training_3_aploss/000000.pth')
Epoch 0/50


/usr/local/lib/python2.7/dist-packages/torch/nn/functional.py:2622: UserWarning: nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample_bilinear is deprecated. Use nn.functional.interpolate instead.")
/home/tomasz/code/dense_correspondence/network/dense_correspondence_network.py:284: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  img_tensor = torch.tensor(img_tensor, device=torch.device("cuda"))


Could not find descriptor image stats...
Only normalizing pairs of images!
Could not find descriptor image stats...
Only normalizing pairs of images!
Could not find descriptor image stats...
Only normalizing pairs of images!
Could not find descriptor image stats...
Only normalizing pairs of images!
Could not find descriptor image stats...
Only normalizing pairs of images!
Could not find descriptor image stats...
Only normalizing pairs of images!
Could not find descriptor image stats...
Only normalizing pairs of images!
Could not find descriptor image stats...
Only normalizing pairs of images!
Could not find descriptor image stats...
Only normalizing pairs of images!
Could not find descriptor image stats...
Only normalizing pairs of images!

 empty data, continuing 


 empty data, continuing 


 empty data, continuing 


 empty data, continuing 


 empty data, continuing 


 empty data, continuing 


 empty data, continuing 


 empty data, continuing 

Could not find descriptor image st

In [ ]:
from torchvision import transforms
import matplotlib.pyplot as plt

denorm = transforms.Compose([ 
    transforms.Normalize(mean=[0,0,0], std=[1./0.229, 1./0.224, 1./0.225]), 
    transforms.Normalize(mean=[-0.485, -0.456, -0.406], std=[1,1,1])
])

def denorm_and_plot(tensor_img):
    tensor_img = tensor_img.clone().detach().requires_grad_(False)
    if tensor_img is None:
        return None
    denom_img = denorm(tensor_img)
    x = denom_img.cpu().detach().numpy()
    return x.transpose(1,2,0)

In [ ]:

from dense_correspondence.evaluation.plotting import normalize_descriptor

In [ ]:
plt.imshow(normalize_descriptor(descriptor[0].detach().cpu().numpy()).transpose(1,2,0))

In [ ]:
img1 = data[1]
img1.shape

In [ ]:
data[-1]

In [ ]:
utils.get_data_dir()

In [ ]:
utils.getPaddedString(12345, width=6)

In [ ]:
img1 = data[2].cpu()
out = denorm_and_plot(img1[0])
plt.imshow(out)

In [ ]:
out * 255


## Evaluate the network quantitatively

This should take ~5 minutes.

In [ ]:
model_folder = os.path.join(logging_dir, name)
model_folder = utils.convert_data_relative_path_to_absolute_path(model_folder)

if EVALUATE:
    DCE = DenseCorrespondenceEvaluation
    num_image_pairs = 100
    DCE.run_evaluation_on_network(model_folder, num_image_pairs=num_image_pairs)      

See `evaluation_quantitative_tutorial.ipynb` for a better place to display the plots.